In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import re

/kaggle/input/aa-reports/df_data.pkl


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments,Trainer, DataCollatorForLanguageModeling, pipeline

tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("sdadas/polish-gpt2-medium")

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/837 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [3]:
from datasets import Dataset,DatasetDict
import pickle

pickle_in = open('/kaggle/input/aa-reports/df_data.pkl', 'rb')
df_fromdict = pickle.load(pickle_in)
pickle_in.close()

df = pd.DataFrame(df_fromdict)
df.head()

,content,num_task,type
0,SIEMENS0S00z0panelem0operatorskim0konfiguracja...,0,instruction
1,Układ1stabilizacji1ciśnienia1sterownik1TURCK1C...,1,instruction
2,Układ2regulacji2temperatury2SIEMENS2Cel2ćwicze...,2,instruction
3,Sterowanie3układem3lewitacji3powietrznej.3Cel3...,3,instruction
4,Konfiguracja4i4uruchomienie4sieci4przemysłowej...,4,instruction


In [4]:
ds = Dataset.from_pandas(df)
context_length = 512

def tokenize(element):
    it_task = 0
    prefix = tokenizer(f"C{element['num_task'][it_task]}; ")['input_ids']
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"][:-1], outputs["input_ids"][:-1]):
        if length != context_length:
            it_task += 1
            prefix = tokenizer(f"C{element['num_task'][it_task]}; ")['input_ids']
            continue
#         input_ids = prefix + input_ids
        input_batch.append(input_ids)
    
    return {"input_ids": input_batch}

dds = ds.train_test_split(0.1, seed=42)
dds = dds.map(
    tokenize, batched=True, remove_columns=dds["train"].column_names
)
dds
# dds['train']['input_ids']

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 95
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 5
    })
})

In [5]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [6]:
bs = 2
lr = 8e-5

def corr_d(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}
tokenizer.pad_token = tokenizer.eos_token

args = TrainingArguments(output_dir='outputs', 
                         learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy="epoch", 
                         per_device_train_batch_size=bs, 
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=10, 
                         weight_decay=0.01, 
                         report_to='none', 
                         gradient_accumulation_steps=4,
                         logging_steps=20,
                         
                         save_total_limit = 2,
                         save_strategy = 'no',
                         load_best_model_at_end=False,
                        )

trainer = Trainer(model,
                  args,
                  data_collator=data_collator,
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  tokenizer=tokenizer)

Using cuda_amp half precision backend


In [7]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [8]:
trainer.args.num_train_epochs = 15
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 95
  Num Epochs = 15
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 180


Epoch,Training Loss,Validation Loss
1,No log,3.790904
2,2.949200,3.696859
3,2.949200,3.741150
4,1.966100,4.013464
5,1.254200,4.206231
6,1.254200,4.384904
7,0.719100,4.499623
8,0.719100,4.602789
9,0.408600,4.725942
10,0.232800,4.782055


***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
***** Running Evaluation *****
  Num examples = 5
  Batch size

TrainOutput(global_step=180, training_loss=0.872678683201472, metrics={'train_runtime': 349.304, 'train_samples_per_second': 4.08, 'train_steps_per_second': 0.515, 'total_flos': 1323398489702400.0, 'train_loss': 0.872678683201472, 'epoch': 15.0})

In [9]:
trainer.save_model(f'/kaggle/working/AA_report_copilot_{trainer.args.num_train_epochs}ep')

Saving model checkpoint to /kaggle/working/AA_report_copilot_15ep
Configuration saved in /kaggle/working/AA_report_copilot_15ep/config.json
Model weights saved in /kaggle/working/AA_report_copilot_15ep/pytorch_model.bin
tokenizer config file saved in /kaggle/working/AA_report_copilot_15ep/tokenizer_config.json
Special tokens file saved in /kaggle/working/AA_report_copilot_15ep/special_tokens_map.json


In [10]:
device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device("cpu")
generator = pipeline(
    "text-generation", 
#     model=AutoModelForCausalLM.from_pretrained('/kaggle/working/AA_report_copilot_v1'),
    model=trainer.model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    device=device,
)

In [11]:
max_len = 50
num_seq = 5
num_task = 6
def generate(input, num_task):
    input_ids = tokenizer.encode(input.replace(' ', str(num_task)), return_tensors='pt').to('cuda')

    outputs = model.generate(
        input_ids, 
        max_length=max_len, 
        no_repeat_ngram_size=2, 
    #     num_beams=10, 
        do_sample=True,
    #     temperature=0.15,
#         top_p=0.95, 
        
#         top_p=0.92, top_k=0,
        penalty_alpha=0.6, top_k=5,
        num_return_sequences=num_seq,
        pad_token_id=tokenizer.pad_token_id,
        device=torch.cuda.current_device()
    )
    print(f"Ćwiczenie {num_task}; {input}...\n" + 100 * '-')
    for i, output in enumerate(outputs):
      print(f"{i}: {re.sub(r'[0-9]', ' ', tokenizer.decode(output, skip_special_tokens=True))}")
    print('')

In [12]:
# new / overfit (20 epochs)
for num_task in range(10):
    generate('Ćwiczenie rozpoczęliśmy od', num_task)

Ćwiczenie 0; Ćwiczenie rozpoczęliśmy od...
----------------------------------------------------------------------------------------------------
0: Ćwiczenie rozpoczęliśmy od przygotowania do ćwiczenia konfiguracji sprzętu. Jako pierwsze podłączone jest moduł SCADA z 
1: Ćwiczenie rozpoczęliśmy od zadania wartości proporcjonalnej do zmiennych użytych wćwiczeniu i odpowiedniego blokowania sygnału wejściowego. Zmienne 
2: Ćwiczenie rozpoczęliśmy od zadania wartości sterowania logicznego a następnie uruchomienia napędu zgodnie ze schematem . Zbudowaliśmy podstawowe
3: Ćwiczenie rozpoczęliśmy od zadania  wartości zadanej i skonfigurowania sprzętu, urządzenia służącego do wizualizacji procesu automatycznej
4: Ćwiczenie rozpoczęliśmy od zadania nazwy zmiennych zgodnych jak d poszczególnych eksperymentów. Na wszystkich etapach wykorzystywaliśmy funk

Ćwiczenie 1; Ćwiczenie rozpoczęliśmy od...
----------------------------------------------------------------------------------------------------
0

In [13]:
# model_name = 'AA_report_copilot_overfit'
# import shutil
# shutil.make_archive(model_name, 'zip', f'/kaggle/working/{model_name}')

# from IPython.display import FileLink
# FileLink(f'{model_name}.zip')